### mpstat jupyter notebook 
To run open in VSCode with a sourced venv via python3 -m venv venvmpstat
or directly from the command line with jupyter notebook and navigate to browser and execute all cells. 
Install packages with 
pip install -r requirements.txt 

I have also tested with a file called short.txt which is a smaller dataset to work with. 
Or you can also see graphs for a particular subset of cpus for example just the all categories for all cpus. 

FYI the large dataset mpstat.ab8713f3-6c19-479c-b662-bad058d8902c.out is 441M and has 2.1 million rows is slow to run.

Sampling just a few particaular cpus can be an option. Randomly selecting ~200k data points from specific cpu core say 2,4,9,200,255 
trimming the dataset is also an option. Using different visualization tools is also an option. 

In [8]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [9]:
# Create a list to hold the JSON data
# The following code only needs to run once
json_data = []

with open('output.json', 'w') as outfile,open("../short.txt","r") as f:
# with open('output-large.json','w') as outfile,open("../mpstat.ab8713f3-6c19-479c-b662-bad058d8902c.out") as f:
     # Split the input data into lines
     headers = []
     for line in f:
         if line is None or line == '\n': 
            continue
         else:
            lines = line.strip().split(' ')
            lines = list(filter(None, lines))
            # Extract the headers
            if len(headers) == 0:
               headers = lines[1:]
               continue
            
            datetime = lines[0]               
            row_dict = { "timestamp": datetime}
                  
            for i in range(len(headers)):
               k = headers[i]
               v = lines[i+1]
               row_dict[k] = v
            json_data.append(row_dict)

     json.dump(json_data, outfile)

In [10]:
with open('output.json', 'r') as infile: 
    data = json.load(infile)
    result = pd.json_normalize(data, errors='raise')
df = pd.DataFrame(result)
df.head()

,timestamp,CPU,%usr,%nice,%sys,%iowait,%irq,%soft,%steal,%guest,%gnice,%idle
0,11:21:38,all,0.34,0.00,8.47,0.04,0.00,0.17,0.00,18.28,0.00,72.64
1,11:21:38,0,0.00,0.00,31.96,0.00,0.00,0.00,0.00,0.00,0.00,68.04
2,11:21:38,1,1.23,0.00,9.88,0.00,0.00,0.00,0.00,0.00,0.00,88.89
3,11:21:38,2,1.06,0.00,1.06,0.00,0.00,0.00,0.00,1.06,0.00,96.81
4,11:21:38,3,0.00,0.00,5.10,0.00,0.00,0.00,0.00,0.00,0.00,94.90


In [30]:
df.describe

<bound method NDFrame.describe of     timestamp  CPU  %usr %nice   %sys %iowait  %irq %soft %steal  %guest  \
0    11:21:38  all  0.34  0.00   8.47    0.04  0.00  0.17   0.00   18.28   
1    11:21:38    0  0.00  0.00  31.96    0.00  0.00  0.00   0.00    0.00   
2    11:21:38    1  1.23  0.00   9.88    0.00  0.00  0.00   0.00    0.00   
3    11:21:38    2  1.06  0.00   1.06    0.00  0.00  0.00   0.00    1.06   
4    11:21:38    3  0.00  0.00   5.10    0.00  0.00  0.00   0.00    0.00   
..        ...  ...   ...   ...    ...     ...   ...   ...    ...     ...   
991  11:21:41  216  0.00  0.00   0.00    0.00  0.00  0.00   0.00  100.00   
992  11:21:41  217  0.00  0.00   0.00    0.00  0.00  0.00   0.00    0.00   
993  11:21:41  218  2.00  0.00   0.00    0.00  0.00  0.00   0.00    0.00   
994  11:21:41  219  0.00  0.00   0.00    0.00  0.00  0.00   0.00   42.86   
995  11:21:41  220  0.00  0.00   0.00    0.00  0.00  0.00   0.00    0.00   

    %gnice   %idle  
0     0.00   72.64  
1     0.00 

In [31]:
df.dtypes

timestamp    object
CPU          object
%usr         object
%nice        object
%sys         object
%iowait      object
%irq         object
%soft        object
%steal       object
%guest       object
%gnice       object
%idle        object
dtype: object

In [32]:
df.rename(columns={'CPU':'cpu', '%usr':'usr','%nice':'nice','%iowait':'iowait','%irq':'irq','%soft':'soft','%steal':'steal','%guest':'guest','%sys':'sys','%gnice':'gnice','%idle':'idle'}, inplace=True)

In [33]:
df.head()

,timestamp,cpu,usr,nice,sys,iowait,irq,soft,steal,guest,gnice,idle
0,11:21:38,all,0.34,0.00,8.47,0.04,0.00,0.17,0.00,18.28,0.00,72.64
1,11:21:38,0,0.00,0.00,31.96,0.00,0.00,0.00,0.00,0.00,0.00,68.04
2,11:21:38,1,1.23,0.00,9.88,0.00,0.00,0.00,0.00,0.00,0.00,88.89
3,11:21:38,2,1.06,0.00,1.06,0.00,0.00,0.00,0.00,1.06,0.00,96.81
4,11:21:38,3,0.00,0.00,5.10,0.00,0.00,0.00,0.00,0.00,0.00,94.90


In [34]:
substring = ['cpu']
df = df[df['cpu'].str.contains("CPU")==False]
df.describe

<bound method NDFrame.describe of     timestamp  cpu   usr  nice    sys iowait   irq  soft steal   guest gnice  \
0    11:21:38  all  0.34  0.00   8.47   0.04  0.00  0.17  0.00   18.28  0.00   
1    11:21:38    0  0.00  0.00  31.96   0.00  0.00  0.00  0.00    0.00  0.00   
2    11:21:38    1  1.23  0.00   9.88   0.00  0.00  0.00  0.00    0.00  0.00   
3    11:21:38    2  1.06  0.00   1.06   0.00  0.00  0.00  0.00    1.06  0.00   
4    11:21:38    3  0.00  0.00   5.10   0.00  0.00  0.00  0.00    0.00  0.00   
..        ...  ...   ...   ...    ...    ...   ...   ...   ...     ...   ...   
991  11:21:41  216  0.00  0.00   0.00   0.00  0.00  0.00  0.00  100.00  0.00   
992  11:21:41  217  0.00  0.00   0.00   0.00  0.00  0.00  0.00    0.00  0.00   
993  11:21:41  218  2.00  0.00   0.00   0.00  0.00  0.00  0.00    0.00  0.00   
994  11:21:41  219  0.00  0.00   0.00   0.00  0.00  0.00  0.00   42.86  0.00   
995  11:21:41  220  0.00  0.00   0.00   0.00  0.00  0.00  0.00    0.00  0.00   

     

In [35]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%H:%M:%S').dt.time

/tmp/ipykernel_17747/2610453095.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
df.dtypes

timestamp    object
cpu          object
usr          object
nice         object
sys          object
iowait       object
irq          object
soft         object
steal        object
guest        object
gnice        object
idle         object
dtype: object

In [37]:
exclude_list = ['all', 'CPU','%usr','%nice','%sys','%iowait','%irq','%soft','%steal','%guest',  '%gnice','%idle']

In [38]:
df.iloc[:,1:]

,cpu,usr,nice,sys,iowait,irq,soft,steal,guest,gnice,idle
0,all,0.34,0.00,8.47,0.04,0.00,0.17,0.00,18.28,0.00,72.64
1,0,0.00,0.00,31.96,0.00,0.00,0.00,0.00,0.00,0.00,68.04
2,1,1.23,0.00,9.88,0.00,0.00,0.00,0.00,0.00,0.00,88.89
3,2,1.06,0.00,1.06,0.00,0.00,0.00,0.00,1.06,0.00,96.81
4,3,0.00,0.00,5.10,0.00,0.00,0.00,0.00,0.00,0.00,94.90
...,...,...,...,...,...,...,...,...,...,...,...
991,216,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
992,217,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00
993,218,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,98.00
994,219,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42.86,0.00,57.14


In [39]:
# df.iloc[:,1:300]
df.iloc[:1,1:-1]

,cpu,usr,nice,sys,iowait,irq,soft,steal,guest,gnice
0,all,0.34,0.00,8.47,0.04,0.00,0.17,0.00,18.28,0.00


In [40]:
# handle all seperately 
substring = ['all']
all_df = df[df['cpu'].str.contains("all")==True]
all_df.describe

<bound method NDFrame.describe of     timestamp  cpu   usr  nice    sys iowait   irq  soft steal  guest gnice  \
0    11:21:38  all  0.34  0.00   8.47   0.04  0.00  0.17  0.00  18.28  0.00   
258  11:21:39  all  0.38  0.00   9.73   0.78  0.00  0.25  0.00  16.74  0.00   
516  11:21:40  all  0.43  0.00   9.29   0.21  0.00  0.15  0.00  16.51  0.00   
774  11:21:41  all  0.31  0.00  10.21   0.09  0.00  0.10  0.00  17.65  0.00   

      idle  
0    72.64  
258  72.04  
516  73.36  
774  71.56  >

In [41]:
df = df[df['cpu'].str.contains("all")==False]
df.head()

,timestamp,cpu,usr,nice,sys,iowait,irq,soft,steal,guest,gnice,idle
1,11:21:38,0,0.00,0.00,31.96,0.00,0.00,0.00,0.00,0.00,0.00,68.04
2,11:21:38,1,1.23,0.00,9.88,0.00,0.00,0.00,0.00,0.00,0.00,88.89
3,11:21:38,2,1.06,0.00,1.06,0.00,0.00,0.00,0.00,1.06,0.00,96.81
4,11:21:38,3,0.00,0.00,5.10,0.00,0.00,0.00,0.00,0.00,0.00,94.90
5,11:21:38,4,0.00,0.00,26.04,0.00,0.00,0.00,0.00,0.00,0.00,73.96


In [42]:
df.describe

<bound method NDFrame.describe of     timestamp  cpu   usr  nice    sys iowait   irq  soft steal   guest gnice  \
1    11:21:38    0  0.00  0.00  31.96   0.00  0.00  0.00  0.00    0.00  0.00   
2    11:21:38    1  1.23  0.00   9.88   0.00  0.00  0.00  0.00    0.00  0.00   
3    11:21:38    2  1.06  0.00   1.06   0.00  0.00  0.00  0.00    1.06  0.00   
4    11:21:38    3  0.00  0.00   5.10   0.00  0.00  0.00  0.00    0.00  0.00   
5    11:21:38    4  0.00  0.00  26.04   0.00  0.00  0.00  0.00    0.00  0.00   
..        ...  ...   ...   ...    ...    ...   ...   ...   ...     ...   ...   
991  11:21:41  216  0.00  0.00   0.00   0.00  0.00  0.00  0.00  100.00  0.00   
992  11:21:41  217  0.00  0.00   0.00   0.00  0.00  0.00  0.00    0.00  0.00   
993  11:21:41  218  2.00  0.00   0.00   0.00  0.00  0.00  0.00    0.00  0.00   
994  11:21:41  219  0.00  0.00   0.00   0.00  0.00  0.00  0.00   42.86  0.00   
995  11:21:41  220  0.00  0.00   0.00   0.00  0.00  0.00  0.00    0.00  0.00   

     

In [43]:
df.iloc[:,1:] = df.iloc[:,1:].astype(float)
df.head()

/tmp/ipykernel_17747/1543458764.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,timestamp,cpu,usr,nice,sys,iowait,irq,soft,steal,guest,gnice,idle
1,11:21:38,0.0,0.0,0.0,31.96,0.0,0.0,0.0,0.0,0.0,0.0,68.04
2,11:21:38,1.0,1.23,0.0,9.88,0.0,0.0,0.0,0.0,0.0,0.0,88.89
3,11:21:38,2.0,1.06,0.0,1.06,0.0,0.0,0.0,0.0,1.06,0.0,96.81
4,11:21:38,3.0,0.0,0.0,5.1,0.0,0.0,0.0,0.0,0.0,0.0,94.9
5,11:21:38,4.0,0.0,0.0,26.04,0.0,0.0,0.0,0.0,0.0,0.0,73.96


In [44]:
df.dtypes

timestamp    object
cpu          object
usr          object
nice         object
sys          object
iowait       object
irq          object
soft         object
steal        object
guest        object
gnice        object
idle         object
dtype: object

In [45]:
# px.line(data_frame=df, x='timestamp', y='idle')

In [46]:
all_df.head()

,timestamp,cpu,usr,nice,sys,iowait,irq,soft,steal,guest,gnice,idle
0,11:21:38,all,0.34,0.00,8.47,0.04,0.00,0.17,0.00,18.28,0.00,72.64
258,11:21:39,all,0.38,0.00,9.73,0.78,0.00,0.25,0.00,16.74,0.00,72.04
516,11:21:40,all,0.43,0.00,9.29,0.21,0.00,0.15,0.00,16.51,0.00,73.36
774,11:21:41,all,0.31,0.00,10.21,0.09,0.00,0.10,0.00,17.65,0.00,71.56


In [11]:
px.line(data_frame=all_df, x='timestamp', y='iowait', title='IOWait over Time')

NameError: name 'all_df' is not defined